In [ ]:
# ✅ Notebook A – Recepção Raw (versão consolidada)
# Função: Upload do Google Drive para o MinIO com auditoria no PostgreSQL

import os
import hashlib
import logging
from pathlib import Path
from datetime import datetime

from minio import Minio
from minio.error import S3Error
import psycopg2
from psycopg2 import sql
from tqdm.notebook import tqdm

import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Configurações ---
GDRIVE_MOUNT_PATH = Path("/home/jovyan/work/gdrive_local_mount/")
MINIO_ENDPOINT = "minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "senhasegura"
MINIO_BUCKET_RAW = "reception-raw"

POSTGRES_HOST = "postgres_db"
POSTGRES_PORT = "5432"
POSTGRES_DB = "postgres"
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "senhasegura"

# --- Logging ---
LOG_DIR = Path("/home/jovyan/work/logs/")
LOG_DIR.mkdir(parents=True, exist_ok=True)
LOG_FILE = LOG_DIR / f"processamento_drive_{datetime.now():%Y%m%d_%H%M%S}.log"

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(LOG_FILE),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)
logger.info("Configurações iniciais carregadas.")

# --- PostgreSQL ---
PG_CONN = psycopg2.connect(
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DB,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD
)
cursor = PG_CONN.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS reception_audit (
        id SERIAL PRIMARY KEY,
        diretorio_origem TEXT NOT NULL,
        caminho_relativo TEXT NOT NULL,
        nome_arquivo TEXT NOT NULL,
        hash_sha256 VARCHAR(64) NOT NULL,
        tamanho_bytes BIGINT NOT NULL,
        data_processamento TIMESTAMPTZ NOT NULL,
        caminho_minio TEXT NOT NULL
    );
""")
PG_CONN.commit()
logger.info("Tabela 'reception_audit' verificada/criada com sucesso.")

# --- MinIO ---
client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)
if not client.bucket_exists(MINIO_BUCKET_RAW):
    client.make_bucket(MINIO_BUCKET_RAW)

# --- Widget de seleção de pasta e botão verde ---
folder_options = []
for root, dirs, _ in os.walk(GDRIVE_MOUNT_PATH):
    for d in dirs:
        rel = os.path.relpath(os.path.join(root, d), GDRIVE_MOUNT_PATH)
        folder_options.append(rel)

folder_dropdown = widgets.Dropdown(options=sorted(folder_options), description="Pasta:")
run_button = widgets.Button(description="INICIAR UPLOAD", button_style='success')  # botão verde
output = widgets.Output()

# --- Função principal com auditoria + barra de progresso ---
def upload_e_auditar(btn):
    with output:
        clear_output()
        selected = folder_dropdown.value
        base_dir = GDRIVE_MOUNT_PATH / selected
        print(f"Iniciando upload da pasta '{selected}' para bucket '{MINIO_BUCKET_RAW}'...")

        tarefas = []
        dados_para_inserir = []

        # Contagem total para barra de progresso
        total_files = sum(len(files) for _, _, files in os.walk(base_dir))
        progresso = tqdm(total=total_files, desc="📤 Upload e auditoria")

        for root, _, files in os.walk(base_dir):
            for fname in files:
                path_local = os.path.join(root, fname)
                rel_drive = os.path.relpath(path_local, GDRIVE_MOUNT_PATH)
                obj_path = os.path.join(selected, os.path.relpath(path_local, base_dir))
                object_name = obj_path.replace(os.sep, "/")

                client.fput_object(MINIO_BUCKET_RAW, object_name, path_local)

                tamanho = os.path.getsize(path_local)
                sha256 = hashlib.sha256()
                with open(path_local, "rb") as f:
                    for chunk in iter(lambda: f.read(8192), b""):
                        sha256.update(chunk)
                hash_hex = sha256.hexdigest()

                dados_para_inserir.append((
                    selected, rel_drive, os.path.basename(path_local),
                    hash_hex, tamanho, datetime.utcnow(), object_name
                ))
                progresso.update(1)

        progresso.close()

        # Inserção em lote no PostgreSQL
        cursor.executemany("""
            INSERT INTO reception_audit (
                diretorio_origem, caminho_relativo, nome_arquivo,
                hash_sha256, tamanho_bytes, data_processamento, caminho_minio
            ) VALUES (%s, %s, %s, %s, %s, %s, %s);
        """, dados_para_inserir)
        PG_CONN.commit()

        print(f"✅ Upload concluído: {len(dados_para_inserir)} arquivos enviados e auditados.")

# --- Conectar widgets à função ---
run_button.on_click(upload_e_auditar)
display(folder_dropdown, run_button, output)
